# Notice:

- waitKey(0)

    - work in .py

    - need to add the cv2.startWindowThread() after openning the startWindowThread
    
    - need to add cv2.destroyAllWindes();  cv2.waitKey(1)

# 1/17: Face Detection w/ web stream| photo| video

[link](https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/)

After opencv 3.3, Ryabnikov includes accurate, deep learning based face detector [face_detector](https://github.com/opencv/opencv/tree/master/samples/dnn/face_detector)

need two file: 

- .prototxt: defines the model architecture

- .caffemodel: contains the weights for the actual layers

### Face Detection for Photo

In [1]:
import numpy as np
import argparse
import cv2

args = {'prototxt': './simple-object-tracking/deploy.prototxt', 
    'model': './simple-object-tracking/res10_300x300_ssd_iter_140000.caffemodel', 
    'image':'./test_1.jpeg', 
    'confidence': .5}

net = cv2.dnn.readNetFromCaffe(args['prototxt'], args['model'])

# load the input image and construct an input blob for the image by resizing to a fixed 300 * 300 pixels and then normalizing it
image = cv2.imread(args['image'])
cv2.startWindowThread()

(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

# pass the blob through the network and obtain the detections and predictions
net.setInput(blob)
detections = net.forward()

# loop over the detections 
for i in range(0, detections.shape[2]): 

    # extract the confidence associated with the prediction
    confidence = detections[0, 0, i, 2]

    # filter out weak detections by ensuring the 'confidence' is greater than the minimum confidence
    if confidence > args['confidence']: 

        # compute the (x, y) - coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype('int')

        # draw the bounding box of the face along with the associated probability
        text = '{:.2f}%'.format(confidence * 100)    
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, .45, (0, 0, 255), 2)



cv2.imshow('Output', image)
cv2.waitKey(0)

cv2.destroyAllWindows()
cv2.waitKey(1)

-1

### Face detection for video

In [2]:
from imutils.video import FPS
import numpy as np
import imutils
import cv2

args = {'prototxt': './simple-object-tracking/deploy.prototxt', 
    'model': './simple-object-tracking/res10_300x300_ssd_iter_140000.caffemodel', 
    'video':'./test.mp4', 
    'confidence': .5}


net = cv2.dnn.readNetFromCaffe(args['prototxt'], args['model'])

# open a pointer to the video stream and start the FPS timer
stream = cv2.VideoCapture(args['video'])
fps = FPS().start()

# loop over frames from the video file stream
while True:

    # grab the frame from the threaded video file stream
    (grabbed, frame) = stream.read()

    # if the frame was not grabbed, then we have reached the end of the stream
    if not grabbed: 
        break

    # resize the frame and covert it to grayscale (while still retaining 3 channels)
    frame = imutils.resize(frame, width = 400)

    # grab the frame dimensions and covert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

    # pass the blob throught the network and obtain the detections and preditions
    net.setInput(blob)
    detections = net.forward()

    #loop over the detections
    for i in range(0, detections.shape[2]): 

        # extract the confidence associated with the prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the 'confidence' is greater than the minimum confidence
        if confidence < args['confidence']: 
            continue
        
        # compute the (x, y)- coordinates of the bounding obx for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype('int')

        # draw the bounding box of the face along with the associated probability
        text = '{:.2f}%'.format(confidence * 100)    
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, .45, (0, 0, 255), 2)

    # show the frame and update the FPS counter
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF
    fps.update()

    # if the 'q' key was pressed, break from the loop
    if key == ord('q'):
        break

# stop the timer and display FPS information
fps.stop()
stream.release()
cv2.destroyAllWindows()


# 2/17 Deal with Images


[link](https://www.pyimagesearch.com/2018/07/19/opencv-tutorial-a-guide-to-learn-opencv/)

- deal cv2 and imutils with basic image skills

- deal cv2 with the erode and mask etc

# Edge Detection




## Holistically-Nested Edge Detection

[link](https://www.pyimagesearch.com/2019/03/04/holistically-nested-edge-detection-with-opencv-and-deep-learning/)

The main method: 

- blob: train the image with `dnn.blobFromImage` function. 

- net: dnn and read from caffe, and register the CropLayer (self defined) into the dv2.dnn_registerLayer

    - the deploy.prototxt and hed_pretrained_bsds.caffemodel

- use the blob into net and get the result

### with Image

In [2]:
class CropLayer(object): 
    def __init__(self, params, blobs): 
        # initialize our starting and ending (x, y) - coordinates of the crop
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0
    
    def getMemoryShapes(self, inputs): 
        # the crop layer will receive two inputs -- we need to crop the first input blob to match the shape of the second one, keeping the batch size and number of channels
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])

        # compute the starting and ending crop coordinates
        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H

        # return the shape of the volume (we will preform the actual crop during the forward pass)
        return [[batchSize, numChannels, H, W]]
    
    def forward(self, inputs): 
        # use the derived (x, y)-coordinates to perform the crop
        return [inputs[0][:, :, self.startY:self.endY, self.startX: self.endX]]

In [2]:
import cv2
import os
test_img = './holistically-nested-edge-detection/images/cat.jpg'
args = {'image': 'test_1.jpeg', 'edge_detector': './holistically-nested-edge-detection/hed_model'}

roots = './holistically-nested-edge-detection/hed_model'
protoPath = os.path.join(args['edge_detector'], 'deploy.prototxt')
modelPath = os.path.join(args['edge_detector'], 'hed_pretrained_bsds.caffemodel')

net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# register our new layer with the model
cv2.dnn_registerLayer('Crop', CropLayer)

# load the input image and grab its dimensions
image = cv2.imread(args['image'])
(H, W) = image.shape[:2]

# convert the image to grayscale, blur it, and perform Canny edge detection
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
canny = cv2.Canny(blurred, 30, 150)

# construct a blob out of the input image for the Holistically-Nested Edge Detector
blob = cv2.dnn.blobFromImage(image, scalefactor = 1.0, size = (W, H), mean = (104.00698793, 116.66876762, 122.67891434), swapRB = False, crop = False)

# set the blob as the input to the network and perform a forward pass to compute the edges
net.setInput(blob)
hed = net.forward()
hed = cv2.resize(hed[0, 0], (W, H))
hed = (255 * hed).astype('uint8')

# show the output edge detection results for Canny and Holistically-Nested Edge Detection
cv2.imshow('Input', image)
cv2.imshow('Canny', canny)
cv2.imshow('HED', hed)
cv2.waitKey(0)

### with Video

In [3]:
import cv2
import os
from imutils.video import FPS
from imutils.video import FileVideoStream
import imutils

video = 'test.mp4'
args = {'input': video, 'edge_detector': './holistically-nested-edge-detection/hed_model'}

roots = './holistically-nested-edge-detection/hed_model'
protoPath = os.path.join(args['edge_detector'], 'deploy.prototxt')
modelPath = os.path.join(args['edge_detector'], 'hed_pretrained_bsds.caffemodel')

# open a pointer to the video stream and start the FPS timer
# vs = cv2.VideoCapture(args['input'])
fvs = FileVideoStream(args['input']).start()
fps = FPS().start()

net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# register our new layer with the model
cv2.dnn_registerLayer('Crop', CropLayer)

# loop over frames from the video stream
# while True:
while fvs.more(): 
    
    # grab the next frame and handle if we are reading from either videocapture or video stream
    # (grabbed, frame) = vs.read()
    # frame = frame[1]
    frame = fvs.read()

    # # if we are viewing a video and we did not grab a frame then we have reached the end of the video
    # if not grabbed:
    #     break

    # load the input image and grab its dimensions
    frame = imutils.resize(frame, width = 500)
    (H, W) = frame.shape[:2]

    # convert the image to grayscale, blur it, and perform Canny edge detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blurred, 30, 150)

    # construct a blob out of the input image for the Holistically-Nested Edge Detector
    blob = cv2.dnn.blobFromImage(frame, scalefactor = 1.0, size = (W, H), mean = (104.00698793, 116.66876762, 122.67891434), swapRB = False, crop = False)

    # set the blob as the input to the network and perform a forward pass to compute the edges
    net.setInput(blob)
    hed = net.forward()
    hed = cv2.resize(hed[0, 0], (W, H))
    hed = (255 * hed).astype('uint8')

    # show the output edge detection results for Canny and Holistically-Nested Edge Detection
    cv2.imshow('Frame', frame)
    cv2.imshow('Canny', canny)
    cv2.imshow('HED', hed)
    key = cv2.waitKey(1) & 0xFF

    fps.update()
    
    # if the 'q' key was pressed, break from the loop
    if key == ord('q'):
        break
fps.stop()
# vs.release()
cv2.destroyAllWindows()
fvs.stop()

## auto Canny edge detection (no parameter)

[link](https://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/)

The main method is to set a sigma, which set as .33 according to author's experience, and set the lower and the upper by +/ - the sigma times the median of the image data (np.median(gray_image)) <- this is already set as the gray one. 

# 3/17 Build Mobile Document Scanner

[link](https://www.pyimagesearch.com/2014/09/01/build-kick-ass-mobile-document-scanner-just-5-minutes/)

code is here -> open_cv_3_17.py

- Detect edges

- use the edges to find contour representing the piece of paper being scanned

- apply perspective transform to obtain the top-down view 

## Some Situation not Suitable

- make sure the shape is CLOSED RECTANGLE

- make sure the background is distinct with the foreground rectangle image. 

## Some adjust suggestion: 

- [use dilate help to make close rectangle](https://stackoverflow.com/questions/43009923/how-to-complete-close-a-contour-in-python-opencv)

- keep in mind, each step could adjust the threshold. So it's better to draw output in each key image. 

- also could add sharpened method before the final output. 

### Detect edges

In [2]:
from pyimagesearch.transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import cv2
import imutils
import sys

args = {'image': 'check_scan_img.JPG'}

# load the image and compute the ratio of the old height to the new height, clone it, and resize it
image = cv2.imread(args['image'])
cv2.startWindowThread()
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)

# convert the image to grayscale, blur it and find edges in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

# this is to use the dilated to make a closed rectangle
# to connect the contour
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
# for kindle pic 5, 5
dilated = cv2.dilate(edged, kernel)


# show the original image and the edge detected image
print ('STEP 1: Edge Detection')
cv2.imshow('Image', image)
cv2.imshow('Edged', edged)
cv2.imshow('Dilated', dilated)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Find Contours

In [ ]:
# find the controus in the edged image, keeping only the largest ones, and initialize the screen contour

cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) # use edged or dilated according to the actual situation
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

# loop over the contours
for ind, c in enumerate(cnts): 
    # approximate the contour
    peri = cv2.arcLength(c, True) # <- 计算封闭周长
    approx = cv2.approxPolyDP(c, .02 * peri, True) # <- 计算多边形有多少条边

    # if our approximated contour has four points, then we can assume that we have found our screen
    if  len(approx) == 4: 
        screenCnt = approx
        break
    
    # if all the cnt is checked but no len(approx) == 4, print the message and quit the python script
    if ind == len(cnts) - 1: 
        print ('no rectangle found, please check the source image. ')
        sys.exit(0)
    
# show the contour (outline) of the piece of paper
print ('STEP 2: Find contours of paper')
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow('Outline', image)
cv2.startWindowThread()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### apply perspective transform to obtain the top-down view

In [ ]:
# apply the four point transform to obtain a top-down view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
warped = add_sharpen_kernel(warped, False) # personal defined

# convert the warped image to grayscale, then threshold it to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

T = threshold_local(warped, 11, offset = 20, method = 'gaussian') # (warped, 11, offset = 10, method = 'gaussian')
warped = (warped > T).astype('uint8') * 255 
# show the original and scanned images
print ('STEP 3: Apply perspective transform')
cv2.imshow('Original', imutils.resize(orig, height = 650))
cv2.imshow('Scanned', imutils.resize(warped, height = 650))

cv2.startWindowThread()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Apply a Perspective Transform & Threshold

In [ ]:
# apply the four point transform to obtain a top-down view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)

# convert the warped image to grayscale, then threshold it to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = 'gaussian')
warped = (warped > T).astype('uint8') * 255

# show the original and scanned images
print ('STEP 3: Apply perspective transform')
cv2.imshow('Original', imutils.resize(orig, height = 650))
cv2.startWindowThread()
cv2.imshow('Scanned', imutils.resize(warped, height = 650))
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

# 4/17 Bubble sheet multiple choice scanner and test grader using OMR (Optical Mark Recognition), etc

[link](https://www.pyimagesearch.com/2016/10/03/bubble-sheet-multiple-choice-scanner-and-test-grader-using-omr-python-and-opencv/)

code is here -> ./open_cv_4_17.py

# 5/17 Ball Tracking

[link](https://www.pyimagesearch.com/2015/09/14/ball-tracking-with-opencv/)

# 6/17 Measure Size of Objects

[link](https://www.pyimagesearch.com/2016/03/28/measuring-size-of-objects-in-an-image-with-opencv/?__s=yo68x506yucrfbb1gtj5)

- get the pixels per metric ratio

- fix the reference object in most left place etc

- 

# 8/19 Facial Detect

Note: 7/19 no special content

[link](https://www.pyimagesearch.com/2017/04/03/facial-landmarks-dlib-opencv-python/?__s=yo68x506yucrfbb1gtj5)

# 9/17 Eye Blink

[link](https://www.pyimagesearch.com/2017/04/24/eye-blink-detection-opencv-python-dlib/?__s=yo68x506yucrfbb1gtj5)

# 10/17 Drowsiness Detection

Notice: 11/17 is a introduction about computer vision, and ads

[link](https://www.pyimagesearch.com/2017/05/08/drowsiness-detection-opencv/)

[raspberry_link](https://www.pyimagesearch.com/2017/10/23/raspberry-pi-facial-landmarks-drowsiness-detection-with-opencv-and-dlib/)

# 12/17 Simple Neural Networks

[link](https://www.pyimagesearch.com/2016/09/26/a-simple-neural-network-with-python-and-keras/)

# 14/17 create custom deep learning datasets 

[link_bing_image_api_built_dataset](https://www.pyimagesearch.com/2018/04/09/how-to-quickly-build-a-deep-learning-image-dataset/)

[googl_image_built_db](https://www.pyimagesearch.com/2017/12/04/how-to-create-a-deep-learning-dataset-using-google-images/)


# 15/17 build a CNN on the custom dataset 

[link](https://www.pyimagesearch.com/2018/04/16/keras-and-convolutional-neural-networks-cnns/)

# 16/17 real-time object detection w/ deep learning 

[link1](https://www.pyimagesearch.com/2017/09/18/real-time-object-detection-with-deep-learning-and-opencv/)

[link2](https://www.pyimagesearch.com/2018/05/14/a-gentle-guide-to-deep-learning-object-detection/)

# 1. Inpainting (图像修复)

[link](https://www.pyimagesearch.com/2020/05/18/image-inpainting-with-opencv-and-python/)

# read barcode

[link](https://www.pyimagesearch.com/2018/05/21/an-opencv-barcode-and-qr-code-scanner-with-zbar/)

# Faster video file FPS w cv2.VideoCapture 

[link](https://www.pyimagesearch.com/2017/02/06/faster-video-file-fps-with-cv2-videocapture-and-opencv/)



In [1]:
class FileVideoStream: 
    def __init__(self, path, queueSize = 128): 
        # initalize the file video stream along with the boolean used to indicate if the thread should be stopped or not
        self.stream = cv2.VideoCapture(path)
        self.stopped = False

        # initialize the queue used to store frames read from the video file
        self.Q = Queue(maxsize = queueSize)
    
    def start(self):
        # start a thread to read frames from the video stream
        t = Thread(target = self.update, args = ())
        t.daemon = True
        t.start()
        return self

    def update(self): 

        # keep looping infinitely
        while True: 
            # if the thread indicator variable is set, stop the thread
            if self.stopped: 
                return 

            # otherwise, ensure the que has room in it
            if not self.Q.full(): 

                # read the next frame from the file
                (grabbed, frame) = self.stream.read()

                # if the grabbed boolean is False, then we have reached the end of the video file
                if not grabbed:
                    self.stop()
                    return 
                
                # add the frame to the queue
                self.Q.put(frame)
    def read(self): 
        # return next frame in the queue
        return self.Q.get()
    
    def more(self): 
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0
    
    def stop(self): 
        # indicate that the thread should be stopped
        self.stopped = True
def auto_canny(image, sigma = .33): 
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed media
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))

    edged = cv2.Canny(image, lower, upper)

    return edged
